# Analysis Template Walkthrough

# Setup

## Select extract
In order for the template cells to query data from the correct repository, enter the repository name as `repository` and repository object type as `object_type`.

In [1]:
repository = 'openml'
object_type = 'datasets'

In [2]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

In [3]:
#see more rows and columns of output
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100) 

## Helper Functions

In [4]:
import os, sys
dir2 = os.path.abspath('../')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

from utils import analysis
from utils.crosswalk import RepositoryExtract, property_crosswalk
from utils import accessors

# Summary Statistic Walkthroughs

Read in repository .json file

In [5]:
df = pd.read_json(f'{repository}_{object_type}.json')

In [6]:
df

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures,page,cache_format,citation,collection_date,contributor,creator,data_feather_file,data_file,data_pickle_file,dataset_id,default_target_attribute,description,feather_attribute_file,features,id,ignore_attribute,language,licence,md5_checksum,openml_url,original_data_url,paper_url,parquet_file,qualities,row_id_attribute,tag,update_comment,upload_date,url,version_label,visibility,num_downloads,num_runs,num_tasks,num_unique_downloads,task_ids,task_types
0,2,anneal,1,1,active,ARFF,684.0,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0,1,pickle,https://archive.ics.uci.edu/ml/citation_policy...,1990,David Sterling and Wray Buntine,"[David Sterling, Wray Buntine]",NaN,/Users/michaelbaluja/.openml/org/openml/www/da...,None,2.0,class,**Author**: Unknown. Donated by David Sterling...,NaN,"{'0': {'LEGAL_DATA_TYPES': ['nominal', 'numeri...",2.0,None,English,Public,4eaed8b6ec9d8211024b6c089b064761,https://www.openml.org/d/2,https://archive.ics.uci.edu/ml/datasets/Annealing,None,None,"{'AutoCorrelation': 0.6064659978, 'CfsSubsetEv...",None,"[study_1, study_14, study_34, study_37, study_...",NaN,2014-04-06T23:19:24,https://www.openml.org/data/v1/download/166687...,1,public,16.0,13779.0,51.0,19.0,"[2, 232, 1766, 1882, 7286, 145952, 1945, 23309...","[Supervised Classification, Supervised Classif..."
1,3,kr-vs-kp,1,1,active,ARFF,1669.0,3.0,1527.0,2.0,37.0,3196.0,0.0,0.0,0.0,37.0,1,pickle,https://archive.ics.uci.edu/ml/citation_policy...,1989-08-01,Rob Holte,Alen Shapiro,NaN,/Users/michaelbaluja/.openml/org/openml/www/da...,None,3.0,class,Author: Alen Shapiro\nSource: [UCI](https://ar...,NaN,"{'0': {'LEGAL_DATA_TYPES': ['nominal', 'numeri...",3.0,None,English,Public,ad6eb32b7492524d4382a40e23cdbb8e,https://www.openml.org/d/3,https://archive.ics.uci.edu/ml/datasets/Chess+...,https://dl.acm.org/doi/abs/10.5555/32231,None,"{'AutoCorrelation': 0.9990610329, 'CfsSubsetEv...",None,"[mythbusting_1, OpenML-CC18, OpenML100, study_...",NaN,2014-04-06T23:19:28,https://www.openml.org/data/v1/download/3/kr-v...,1,public,44.0,274200.0,41.0,56.0,"[3, 145953, 1767, 233, 1883, 317602, 233091, 1...","[Supervised Classification, Supervised Classif..."
2,4,labor,1,1,active,ARFF,37.0,3.0,20.0,2.0,17.0,57.0,56.0,326.0,8.0,9.0,1,pickle,https://archive.ics.uci.edu/ml/citation_policy...,1988-11-01,Stan Matwin,Collective Bargaining Review of Labour Canada,NaN,/Users/michaelbaluja/.openml/org/openml/www/da...,None,4.0,class,**Author**: Unknown\n**Source**: Collective Ba...,NaN,"{'0': {'LEGAL_DATA_TYPES': ['nominal', 'numeri...",4.0,None,English,Public,e013a5e0acb7e141d169198a55b22c51,https://www.openml.org/d/4,https://archive.ics.uci.edu/ml/datasets/Labor+...,http://citeseerx.ist.psu.edu/viewdoc/summary?d...,None,"{'AutoCorrelation': 0.75, 'CfsSubsetEval_Decis...",None,"[mythbusting_1, study_1, study_15, study_20, s...",NaN,2014-04-06T23:19:30,https://www.openml.org/data/v1/download/4/labo...,1,public,17.0,7681.0,29.0,17.0,"[4, 1768, 234, 1884, 145954, 1946, 64, 1703, 1...","[Supervised Classification, Supervised Classif..."
3,5,arrhythmia,1,1,active,ARFF,245.0,13.0,2.0,13.0,280.0,452.0,384.0,408.0,206.0,74.0,1,pickle,https://archive.ics.uci.edu/ml/citation_policy...,1998-01-01,None,"[H. Altay Guvenir, Burak Acar, Haldun Muderris...",NaN,/Users/michaelbaluja/.openml/org/openml/www/da...,None,5.0,class,"**Author**: H. Altay Guvenir, Burak Acar, Hald...",NaN,"{'0': {'LEGAL_DATA_TYPES': ['nominal', 'numeri...",5.0,None,None,Public,16b2e6fb5105155638cde38b2d117fac,https://www.openml.org/d/5,https://archive.ics.uci.edu/ml/datasets/arrhyt...,None,None,"{'AutoCorrelation': 0.354767184, 'CfsSubsetEva...",None,"[sport, study_1, study_41, study_76, study_93,...",NaN,2014-04-06T23:19:36,https://www.openml.org/data/v1/download/5/arrh

## 1. How many total objects (not just records) are in our main dataset extracts for each repository?
**Property:** unique_identifier

In [7]:
ids = df.OpenMLDatasetsCrosswalk.unique_identifier

In [8]:
ids

0           2
1           3
2           4
3           5
4           6
        ...  
3458    43119
3459    43121
3460    43122
3461    43123
3462    43127
Name: did, Length: 3463, dtype: int64

In [9]:
ids.nunique()

3461

In [10]:
print(f'There are {len(ids)} items in the OpenML extract, with {ids.nunique()} unique IDs.')

There are 3463 items in the OpenML extract, with 3461 unique IDs.


In [11]:
#for the most part, each row is a unique object, except for two with duplicates
ids.value_counts()

1179     2
1178     2
4097     1
4084     1
4074     1
        ..
1471     1
1472     1
1473     1
1475     1
43127    1
Name: did, Length: 3461, dtype: int64

In [12]:
#check this duplicate object
dupes = df[df['did'] == 1179]
dupes

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures,page,cache_format,citation,collection_date,contributor,creator,data_feather_file,data_file,data_pickle_file,dataset_id,default_target_attribute,description,feather_attribute_file,features,id,ignore_attribute,language,licence,md5_checksum,openml_url,original_data_url,paper_url,parquet_file,qualities,row_id_attribute,tag,update_comment,upload_date,url,version_label,visibility,num_downloads,num_runs,num_tasks,num_unique_downloads,task_ids,task_types
1003,1179,BNG(solar-flare),1,1,active,ARFF,994382.0,8.0,1393.0,3.0,13.0,1000000.0,0.0,0.0,0.0,13.0,1,pickle,None,2014-11-12 12:13:19,None,"[Geoffrey Holmes, Bernhard Pfahringer, Jan van...",NaN,/Users/michaelbaluja/.openml/org/openml/www/da...,None,1178.0,X-class_flares_production_by_this_region,None,NaN,"{'0': {'LEGAL_DATA_TYPES': ['nominal', 'numeri...",1178.0,None,None,public domain,5dc14d82580d27f169aecdb1fc4b07ca,https://www.openml.org/d/1178,None,None,None,"{'AutoCorrelation': 0.9550464094000001, 'CfsSu...",None,"[artificial, BNG]",NaN,2014-11-12T12:13:14,https://www.openml.org/data/v1/download/150656...,None,public,3.0,90.0,22.0,3.0,"[146087, 146385, 168071, 167713, 167355, 16848...","[Supervised Classification, Learning Curve, Le..."
1004,1179,BNG(solar-flare),1,1,active,ARFF,994382.0,8.0,1393.0,3.0,13.0,1000000.0,0.0,0.0,0.0,13.0,1,pickle,None,2014-11-12 12:13:28,None,"[Geoffrey Holmes, Bernhard Pfahringer, Jan van...",NaN,/Users/michaelbaluja/.openml/org/openml/www/da...,None,1179.0,X-class_flares_production_by_this_region,None,NaN,"{'0': {'LEGAL_DATA_TYPES': ['nominal', 'numeri...",1179.0,None,None,public domain,5d9fdf8c44970fc782cb21ef722bd236,https://www.openml.org/d/1179,None,None,None,"{'AutoCorrelation': 0.9888189888000001, 'CfsSu...",None,"[artificial, BNG, study_16]",NaN,2014-11-12T12:13:24,https://www.openml.org/data/v1/download/150657...,None,public,2.0,337.0,22.0,2.0,"[146088, 146386, 168072, 167714, 167356, 16848...","[Supervised Classification, Learning Curve, Le..."


In [13]:
dupes1 = dupes.iloc[0]
dupes2 = dupes.iloc[1]

In [14]:
dupes1

did                                                                                1179
name                                                                   BNG(solar-flare)
version                                                                               1
uploader                                                                              1
status                                                                           active
format                                                                             ARFF
MajorityClassSize                                                              994382.0
MaxNominalAttDistinctValues                                                         8.0
MinorityClassSize                                                                1393.0
NumberOfClasses                                                                     3.0
NumberOfFeatures                                                                   13.0
NumberOfInstances               

In [15]:
dupes2

did                                                                                1179
name                                                                   BNG(solar-flare)
version                                                                               1
uploader                                                                              1
status                                                                           active
format                                                                             ARFF
MajorityClassSize                                                              994382.0
MaxNominalAttDistinctValues                                                         8.0
MinorityClassSize                                                                1393.0
NumberOfClasses                                                                     3.0
NumberOfFeatures                                                                   13.0
NumberOfInstances               

In [16]:
#quite a few differences here, but many of these are None of NaN

In [17]:
dupes1_clean = dupes1.dropna()
dupes2_clean = dupes2.dropna()

In [18]:
#find which fields have real differences
openml_diffs = dupes1_clean == dupes2_clean
openml_diffs

did                                    True
name                                   True
version                                True
uploader                               True
status                                 True
format                                 True
MajorityClassSize                      True
MaxNominalAttDistinctValues            True
MinorityClassSize                      True
NumberOfClasses                        True
NumberOfFeatures                       True
NumberOfInstances                      True
NumberOfInstancesWithMissingValues     True
NumberOfMissingValues                  True
NumberOfNumericFeatures                True
NumberOfSymbolicFeatures               True
page                                   True
cache_format                           True
collection_date                       False
creator                                True
data_file                             False
dataset_id                            False
default_target_attribute        

In [19]:
index_diffs = openml_diffs[openml_diffs == False]
index_diffs

collection_date         False
data_file               False
dataset_id              False
id                      False
md5_checksum            False
openml_url              False
qualities               False
tag                     False
upload_date             False
url                     False
num_downloads           False
num_runs                False
num_unique_downloads    False
task_ids                False
dtype: bool

In [20]:
index_diffs_str = index_diffs.index.tolist()
index_diffs_str

['collection_date',
 'data_file',
 'dataset_id',
 'id',
 'md5_checksum',
 'openml_url',
 'qualities',
 'tag',
 'upload_date',
 'url',
 'num_downloads',
 'num_runs',
 'num_unique_downloads',
 'task_ids']

In [21]:
dupes1_clean[index_diffs_str]

collection_date                                       2014-11-12 12:13:19
data_file               /Users/michaelbaluja/.openml/org/openml/www/da...
dataset_id                                                         1178.0
id                                                                 1178.0
md5_checksum                             5dc14d82580d27f169aecdb1fc4b07ca
openml_url                                  https://www.openml.org/d/1178
qualities               {'AutoCorrelation': 0.9550464094000001, 'CfsSu...
tag                                                     [artificial, BNG]
upload_date                                           2014-11-12T12:13:14
url                     https://www.openml.org/data/v1/download/150656...
num_downloads                                                         3.0
num_runs                                                             90.0
num_unique_downloads                                                  3.0
task_ids                [146087, 14638

In [22]:
dupes2_clean[index_diffs_str]

collection_date                                       2014-11-12 12:13:28
data_file               /Users/michaelbaluja/.openml/org/openml/www/da...
dataset_id                                                         1179.0
id                                                                 1179.0
md5_checksum                             5d9fdf8c44970fc782cb21ef722bd236
openml_url                                  https://www.openml.org/d/1179
qualities               {'AutoCorrelation': 0.9888189888000001, 'CfsSu...
tag                                           [artificial, BNG, study_16]
upload_date                                           2014-11-12T12:13:24
url                     https://www.openml.org/data/v1/download/150657...
num_downloads                                                         2.0
num_runs                                                            337.0
num_unique_downloads                                                  2.0
task_ids                [146088, 14638

In [23]:
#we can ignore data_file, which is artifact of data collection for this project
#looks like collection and upload date change minimally
#appears to be re-upload - download URL leads to file and file (1) [so duplicate]

In [24]:
#subset to view only the duplicate ids
dupes_all = ids.value_counts().to_frame()
dupes_all = dupes_all[dupes_all['did'] == 2]
dupes_all

,did
1179,2
1178,2


In [25]:
dupes_all_ids = dupes_all.index.to_list()
dupes_all_ids

[1179, 1178]

In [26]:
dupes_df = df[df.id.isin(dupes_all_ids)]
dupes_df[index_diffs_str].sort_values('upload_date')

,collection_date,data_file,dataset_id,id,md5_checksum,openml_url,qualities,tag,upload_date,url,num_downloads,num_runs,num_unique_downloads,task_ids
1001,2014-11-12 12:13:19,/Users/michaelbaluja/.openml/org/openml/www/da...,1178.0,1178.0,5dc14d82580d27f169aecdb1fc4b07ca,https://www.openml.org/d/1178,"{'AutoCorrelation': 0.9550464094000001, 'CfsSu...","[artificial, BNG]",2014-11-12T12:13:14,https://www.openml.org/data/v1/download/150656...,3.0,90.0,3.0,"[146087, 146385, 168071, 167713, 167355, 16848..."
1003,2014-11-12 12:13:19,/Users/michaelbaluja/.openml/org/openml/www/da...,1178.0,1178.0,5dc14d82580d27f169aecdb1fc4b07ca,https://www.openml.org/d/1178,"{'AutoCorrelation': 0.9550464094000001, 'CfsSu...","[artificial, BNG]",2014-11-12T12:13:14,https://www.openml.org/data/v1/download/150656...,3.0,90.0,3.0,"[146087, 146385, 168071, 167713, 167355, 16848..."
1002,2014-11-12 12:13:28,/Users/michaelbaluja/.openml/org/openml/www/da...,1179.0,1179.0,5d9fdf8c44970fc782cb21ef722bd236,https://www.openml.org/d/1179,"{'AutoCorrelation': 0.9888189888000001, 'CfsSu...","[artificial, BNG, study_16]",2014-11-12T12:13:24,https://www.openml.org/data/v1/download/150657...,2.0,337.0,2.0,"[146088, 146386, 168072, 167714, 167356, 16848..."
1004,2014-11-12 12:13:28,/Users/michaelbaluja/.openml/org/openml/www/da...,1179.0,1179.0,5d9fdf8c44970fc782cb21ef722bd236,https://www.openml.org/d/1179,"{'AutoCorrelation': 0.9888189888000001, 'CfsSu...","[artificial, BNG, study_16]",2014-11-12T12:13:24,https://www.openml.org/data/v1/download/150657...,2.0,337.0,2.0,"[146088, 146386, 168072, 167714, 167356, 16848..."


In [27]:
#two latter uploaded objects have different did values
dupes_df[['did', 'name', 'collection_date', 'upload_date']].sort_values('upload_date')

,did,name,collection_date,upload_date
1001,1178,BNG(solar-flare),2014-11-12 12:13:19,2014-11-12T12:13:14
1003,1179,BNG(solar-flare),2014-11-12 12:13:19,2014-11-12T12:13:14
1002,1178,BNG(solar-flare),2014-11-12 12:13:28,2014-11-12T12:13:24
1004,1179,BNG(solar-flare),2014-11-12 12:13:28,2014-11-12T12:13:24


In [28]:
#all 4 of these are actually duplications of a single object

In [29]:
#decision - remove object 1178 and keep latter upload date row for object 1179

In [30]:
len(df)

3463

In [31]:
#remove object 1178
df_use = df[df.did != 1178]

In [32]:
#confirm new number of ids
len(df_use)

3461

In [33]:
#group by did, sort descending by upload_date, and select first within group
df_use = df_use.sort_values(['upload_date'], ascending = False).groupby('did').nth(0).reset_index()
df_use[df_use.did == 1179][['did','collection_date', 'upload_date']]

,did,collection_date,upload_date
1001,1179,2014-11-12 12:13:28,2014-11-12T12:13:24


In [34]:
#confirm new number of ids
len(df_use)

3460

In [35]:
#for matching other notebooks, rename 'df_use' back to 'df'
df = df_use

In [36]:
ids = df.OpenMLDatasetsCrosswalk.unique_identifier
print(f'There are {len(ids)} items in the OpenML extract, with {ids.nunique()} unique IDs.')

There are 3460 items in the OpenML extract, with 3460 unique IDs.


## 2. See the "Licenses offered" tab in /Working documents/Licenses sheet for list of licenses by repo.

## Given the type(s) of license(s) offered by the repo, how many of each type is assigned?
**Property:** License

In [37]:
licenses = df.OpenMLDatasetsCrosswalk.license
licenses

0                        Public
1                        Public
2                        Public
3                        Public
4                        Public
                 ...           
3455                        CC0
3456                        CC0
3457                        CC0
3458    BSD (from scikit-learn)
3459    BSD (from scikit-learn)
Name: licence, Length: 3460, dtype: object

In [38]:
license_counts = licenses.value_counts().to_frame()
license_counts['percent'] = license_counts['licence']/len(licenses)*100
license_counts

,licence,percent
Public,2802,80.982659
CC0,275,7.947977
public domain,98,2.832370
Free,53,1.531792
Publicly available,43,1.242775
public,41,1.184971
CC-BY,16,0.462428
CC_BY-SA,10,0.289017
Undefined,10,0.289017
BSD (from scikit-learn),10,0.289017


## 3. What is the mean number of characters (excluding whitespaces, if possible) per object?
**Property:** Description
**Related function:** `mean_characters`

In [39]:
descriptions = df.OpenMLDatasetsCrosswalk.description
descriptions

0       **Author**: Unknown. Donated by David Sterling...
1       Author: Alen Shapiro\nSource: [UCI](https://ar...
2       **Author**: Unknown\n**Source**: Collective Ba...
3       **Author**: H. Altay Guvenir, Burak Acar, Hald...
4       **Author**: David J. Slate  \n**Source**: [UCI...
                              ...                        
3455                                            whitewine
3456    2nd Place Lightgbm Solution of Kaggle Porto Se...
3457    Rotating hyperplane is a stream generator that...
3458    .. _diabetes_dataset:\n\nDiabetes dataset\n---...
3459                                      une description
Name: description, Length: 3460, dtype: object

In [40]:
#text cleaning needed for more accuracte counts, but this is a starting estimate for mean characters
print(f'{analysis.mean_characters(descriptions)} mean characters')

1211.3025814642403 mean characters


## 4. What is the mean number of characters (excluding whitespaces, if possible) per object?
**Property:** Methods
**Related function:** `mean_characters`

In [41]:
methods = df.OpenMLDatasetsCrosswalk.methods
methods

In [42]:
#confirm missing for this repo
print(df.OpenMLDatasetsCrosswalk.methods)

None


## 5. What are the min and max publication dates for each repo?

## How many objects were published each year for each repo?
**Property:** Publication date

In [43]:
publication_dates = df.OpenMLDatasetsCrosswalk.publication_date
publication_dates

0       2014-04-06T23:19:24
1       2014-04-06T23:19:28
2       2014-04-06T23:19:30
3       2014-04-06T23:19:36
4       2014-04-06T23:19:41
               ...         
3455    2021-11-12T23:49:39
3456    2021-12-05T07:54:08
3457    2021-12-08T14:02:06
3458    2021-12-09T13:15:11
3459    2021-12-15T17:36:49
Name: upload_date, Length: 3460, dtype: object

In [44]:
#remove None values
publication_dates = pd.Series(filter(None, publication_dates))

In [45]:
#min and max publication year
publication_dates.min(), publication_dates.max()

('2014-04-06T23:19:20', '2021-12-15T17:36:49')

In [46]:
#records per year
publication_dates.astype('datetime64').apply(lambda date: date.year).value_counts().sort_index()

2014    1036
2015    1323
2016      55
2017      95
2018      41
2019     422
2020     253
2021     192
dtype: int64

In [47]:
#export for plotting
pub_dates_export = publication_dates.astype('datetime64').apply(lambda date: date.year).value_counts().sort_index().to_frame()

In [48]:
#update column names
pub_dates_export_ready = pub_dates_export.reset_index(level=0)
pub_dates_export_ready.columns = ['year', 'count']

In [49]:
#add column with name of repo
pub_dates_export_ready['repo'] = 'openml'
pub_dates_export_ready

,year,count,repo
0,2014,1036,openml
1,2015,1323,openml
2,2016,55,openml
3,2017,95,openml
4,2018,41,openml
5,2019,422,openml
6,2020,253,openml
7,2021,192,openml


In [50]:
#export to Figures folder
pub_dates_export_ready.to_csv('..\\..\\Figures\\Figure1\\repository_dates\\openml_pub_years.csv')

## 6. What are the unweighted mean, median, and max file sizes among all ingested files?
**Property:** File size
**Related function:** `get_summary_statistics`

We first get the file size attribute using the crosswalk.

In [51]:
file_sizes = df.OpenMLDatasetsCrosswalk.file_size
file_sizes

In [52]:
#confirm missing for this repo
print(df.OpenMLDatasetsCrosswalk.file_size)

None


## 7. What are the mean, median, and max number of files per object?
**Property:** URL
**Related function:** `get_summary_statistics`

In [53]:
files = df.OpenMLDatasetsCrosswalk.url
files

0       https://www.openml.org/data/v1/download/166687...
1       https://www.openml.org/data/v1/download/3/kr-v...
2       https://www.openml.org/data/v1/download/4/labo...
3       https://www.openml.org/data/v1/download/5/arrh...
4       https://www.openml.org/data/v1/download/6/lett...
                              ...                        
3455    https://www.openml.org/data/v1/download/220479...
3456    https://www.openml.org/data/v1/download/221015...
3457    https://www.openml.org/data/v1/download/221015...
3458    https://www.openml.org/data/v1/download/221016...
3459    https://www.openml.org/data/v1/download/221016...
Name: url, Length: 3460, dtype: object

In [54]:
files = files.dropna()
files

0       https://www.openml.org/data/v1/download/166687...
1       https://www.openml.org/data/v1/download/3/kr-v...
2       https://www.openml.org/data/v1/download/4/labo...
3       https://www.openml.org/data/v1/download/5/arrh...
4       https://www.openml.org/data/v1/download/6/lett...
                              ...                        
3455    https://www.openml.org/data/v1/download/220479...
3456    https://www.openml.org/data/v1/download/221015...
3457    https://www.openml.org/data/v1/download/221015...
3458    https://www.openml.org/data/v1/download/221016...
3459    https://www.openml.org/data/v1/download/221016...
Name: url, Length: 3417, dtype: object

In [55]:
#files are strings; convert to list to ensure one string URL per object for file

In [56]:
files_list = files.str.split(',')
files_list

0       [https://www.openml.org/data/v1/download/16668...
1       [https://www.openml.org/data/v1/download/3/kr-...
2       [https://www.openml.org/data/v1/download/4/lab...
3       [https://www.openml.org/data/v1/download/5/arr...
4       [https://www.openml.org/data/v1/download/6/let...
                              ...                        
3455    [https://www.openml.org/data/v1/download/22047...
3456    [https://www.openml.org/data/v1/download/22101...
3457    [https://www.openml.org/data/v1/download/22101...
3458    [https://www.openml.org/data/v1/download/22101...
3459    [https://www.openml.org/data/v1/download/22101...
Name: url, Length: 3417, dtype: object

In [57]:
files_counts = files_list.apply(len)
files_counts.sort_values(ascending = False)

1104    3
64      3
95      3
94      3
1086    3
       ..
1215    1
1216    1
1217    1
1218    1
3459    1
Name: url, Length: 3417, dtype: int64

In [58]:
#confirm that those 3 values are accurate
print(files[1104])
print(files[64])

https://www.openml.org/data/v1/download/584452/BNG(autos,10000,10).arff
https://www.openml.org/data/v1/download/1836/BNG(bridges_version1,nominal,1000000).arff


In [59]:
#incorrect - these are also single files they just have commas in file name

In [60]:
#because we know from preliminary analylsis that all OpenML obects are ARFF file format
#makes sense that all objects have just one associated file

All objects have only one associated file

## 8. What are the mean, median, and max total dataset size (summed across all files) per object?
**Property:** Dataset size
**Related function:** `get_summary_statistics`

In [61]:
dataset_sizes = df.OpenMLDatasetsCrosswalk.dataset_size
dataset_sizes

In [62]:
#confirm missing for this repo
print(df.OpenMLDatasetsCrosswalk.dataset_size)

None


## 9. How many of each scientific domain are assigned?
**Property:** Domain
**Related function:** `domains.value_counts()`

In [63]:
domains = df.OpenMLDatasetsCrosswalk.domain
domains

In [64]:
#confirm missing for repo
print(df.OpenMLDatasetsCrosswalk.domain)

None


## 10. What is the mean number of characters (excluding whitespaces, if possible) per object?
**Property:** Technical details
**Related function:** `mean_characters`

In [65]:
# "usage notes" is not in crosswalk

## 11-13. What are the mean and median total number of keyword terms per object, after merging results for Keyword, Geographic keyword, and Scientific keyword?
**Property:** Keyword

In [66]:
print(df.OpenMLDatasetsCrosswalk.keyword)

0       [study_1, study_14, study_34, study_37, study_...
1       [mythbusting_1, OpenML-CC18, OpenML100, study_...
2       [mythbusting_1, study_1, study_15, study_20, s...
3       [sport, study_1, study_41, study_76, study_93,...
4       [AzurePilot, AzurePilot1, OpenML-CC18, OpenML1...
                              ...                        
3455                                                 None
3456                                                 None
3457                                        concept drift
3458                                                 None
3459                                                 None
Name: tag, Length: 3460, dtype: object


In [67]:
print(df.OpenMLDatasetsCrosswalk.geographic_keyword)

None


In [68]:
print(df.OpenMLDatasetsCrosswalk.scientific_keyword)

None


In [69]:
keywords = df.OpenMLDatasetsCrosswalk.keyword
keywords

0       [study_1, study_14, study_34, study_37, study_...
1       [mythbusting_1, OpenML-CC18, OpenML100, study_...
2       [mythbusting_1, study_1, study_15, study_20, s...
3       [sport, study_1, study_41, study_76, study_93,...
4       [AzurePilot, AzurePilot1, OpenML-CC18, OpenML1...
                              ...                        
3455                                                 None
3456                                                 None
3457                                        concept drift
3458                                                 None
3459                                                 None
Name: tag, Length: 3460, dtype: object

In [70]:
#replace the None values with empty lists so the count of string values evaluates to 0
#and make sure each cell is a list
keywords_use = keywords.to_frame().apply(
    lambda row: row.apply(
        lambda cell: [cell] if isinstance(cell, str) else cell if isinstance(cell, list) else []
    ),
    axis=1    
)
keywords_use

,tag
0,"[study_1, study_14, study_34, study_37, study_..."
1,"[mythbusting_1, OpenML-CC18, OpenML100, study_..."
2,"[mythbusting_1, study_1, study_15, study_20, s..."
3,"[sport, study_1, study_41, study_76, study_93,..."
4,"[AzurePilot, AzurePilot1, OpenML-CC18, OpenML1..."
...,...
3455,[]
3456,[]
3457,[concept drift]
3458,[]


In [71]:
#count keywords for each object
keyword_counts = keywords_use.apply(
    lambda row: sum([len(row[entry]) for entry in keywords_use.columns if row[entry]]),
    axis=1
)
keyword_counts

0        9
1       20
2        7
3        6
4       16
        ..
3455     0
3456     0
3457     1
3458     0
3459     0
Length: 3460, dtype: int64

In [72]:
#get summary statistics
analysis.get_summary_statistics(keyword_counts)

{'mean': 2.421098265895954, 'median': 2.0, 'max': 23}

## 14. Who are the most common funding agencies for each repo? What are the object counts per agency?
**Property:** Funding Agency

In [73]:
funders = df.OpenMLDatasetsCrosswalk.funding_agency
funders

In [74]:
#confirm missing for this repo
print(df.OpenMLDatasetsCrosswalk.funding_agency)

None


## 15. What are the mean, median, and max number of Views per object?
**Property:** Views
**Related function:** `get_summary_statistics`

In [75]:
views = df.OpenMLDatasetsCrosswalk.views
views

In [76]:
#confirm missing for repo
print(df.OpenMLDatasetsCrosswalk.views)

None


## 16. What are the mean, median, and max (total) number of downloads per object?
**Property:** Downloads
**Related function:** `get_summary_statistics`

In [77]:
downloads = df.OpenMLDatasetsCrosswalk.downloads
downloads

,num_downloads,num_unique_downloads
0,16.0,19.0
1,44.0,56.0
2,17.0,17.0
3,50.0,57.0
4,73.0,89.0
...,...,...
3455,0.0,0.0
3456,0.0,0.0
3457,0.0,0.0
3458,0.0,0.0


In [78]:
#number of downloads
analysis.get_summary_statistics(downloads['num_downloads'].dropna())

{'mean': 3.4796605209247877, 'median': 1.0, 'max': 294.0}

In [79]:
#number of unique downloads
analysis.get_summary_statistics(downloads['num_unique_downloads'].dropna())

{'mean': 4.309628328943518, 'median': 1.0, 'max': 457.0}

In [80]:
#max unique shouldn't be higher than max for all downloads
downloads.loc[downloads['num_downloads'] == 294]

,num_downloads,num_unique_downloads
26,294.0,457.0


In [81]:
#this is from the underlying data, not something induced by analysis

## 17. What are the mean, median, and max Citation counts per object?
**Property:** Citation count
**Related function:** `get_summary_statistics`

In [82]:
citation_count = df.OpenMLDatasetsCrosswalk.citation_count
citation_count

In [83]:
#confirm missing for repo
print(df.OpenMLDatasetsCrosswalk.citation_count)

None


## 18. How many objects contain each given resource type?
**Property:** Resource type

In [84]:
resource_types = df.OpenMLDatasetsCrosswalk.resource_type
resource_types

In [85]:
#confirm missing for repo
print(df.OpenMLDatasetsCrosswalk.resource_type)

None


## 19. How many objects contain each type of file extension given?
**Property:** File Extension
**Related function:** `get_file_extensions`

In [86]:
files = df.OpenMLDatasetsCrosswalk.file_extension
files

In [87]:
#confirm missing for this repo
print(df.OpenMLDatasetsCrosswalk.file_extension)

None


## 19.5 How many files of each type of file extension are present?
**Property:** File extension

In [88]:
files = df.OpenMLDatasetsCrosswalk.file_extension
files

In [89]:
#confirm missing for this repo
print(df.OpenMLDatasetsCrosswalk.file_extension)

None


## 20. How many objects contain each type of File format given?
**Property:** File format

In [90]:
file_formats = df.OpenMLDatasetsCrosswalk.file_format
file_formats

0       ARFF
1       ARFF
2       ARFF
3       ARFF
4       ARFF
        ... 
3455    ARFF
3456    ARFF
3457    ARFF
3458    arff
3459    arff
Name: format, Length: 3460, dtype: object

In [91]:
file_formats.value_counts()

ARFF           1815
Sparse_ARFF    1149
arff            495
sparse_arff       1
Name: format, dtype: int64

All objects have ARFF format

## 21. How many objects contain each type of Media type given?
**Property:** Media type

In [92]:
media_types = df.OpenMLDatasetsCrosswalk.media_type
media_types

In [93]:
#confirm missing for repo
print(df.OpenMLDatasetsCrosswalk.media_type)

None


## 22. a) How many objects report one related resource type, and b) how many objects report each of those types? c) How many objects report multiple related resource types (regardless of which types)?
**Property:** Related resource type

In [94]:
related_resource_types = df.OpenMLDatasetsCrosswalk.related_resource_type
related_resource_types

In [95]:
#confirm missing for repo
print(df.OpenMLDatasetsCrosswalk.related_resource_type)

None



## 23-25. If there is an entry for an object in one of the three properties (Original data URL, Primary manuscript PID/URL, and Related resource identifier) count as Related resources = True and then count the number of objects that return True.
**Property:** Related Resource Identifier

In [96]:
print(df.OpenMLDatasetsCrosswalk.original_data_url)

0       https://archive.ics.uci.edu/ml/datasets/Annealing
1       https://archive.ics.uci.edu/ml/datasets/Chess+...
2       https://archive.ics.uci.edu/ml/datasets/Labor+...
3       https://archive.ics.uci.edu/ml/datasets/arrhyt...
4       https://archive.ics.uci.edu/ml/datasets/Letter...
                              ...                        
3455                                                 None
3456    https://www.kaggle.com/xiaozhouwang/2nd-place-...
3457                                                 None
3458    https://www4.stat.ncsu.edu/~boos/var.select/di...
3459    https://www4.stat.ncsu.edu/~boos/var.select/di...
Name: original_data_url, Length: 3460, dtype: object


In [97]:
print(df.OpenMLDatasetsCrosswalk.primary_manuscript)

None


In [98]:
print(df.OpenMLDatasetsCrosswalk.related_resource_identifier)

0                                                    None
1                https://dl.acm.org/doi/abs/10.5555/32231
2       http://citeseerx.ist.psu.edu/viewdoc/summary?d...
3                                                    None
4       https://link.springer.com/article/10.1007%252F...
                              ...                        
3455                                                 None
3456                                                 None
3457                                                 None
3458    https://web.stanford.edu/~hastie/Papers/LARS/L...
3459                                                 test
Name: paper_url, Length: 3460, dtype: object


In [99]:
related_resource1 = df.OpenMLDatasetsCrosswalk.original_data_url
related_resource2 = df.OpenMLDatasetsCrosswalk.related_resource_identifier

In [100]:
#concatenate resources and id
related_resource_all = pd.concat([related_resource1, related_resource2], axis = 1)
related_resource_all = pd.concat([ids, related_resource_all], axis = 1)
related_resource_all

,did,original_data_url,paper_url
0,2,https://archive.ics.uci.edu/ml/datasets/Annealing,None
1,3,https://archive.ics.uci.edu/ml/datasets/Chess+...,https://dl.acm.org/doi/abs/10.5555/32231
2,4,https://archive.ics.uci.edu/ml/datasets/Labor+...,http://citeseerx.ist.psu.edu/viewdoc/summary?d...
3,5,https://archive.ics.uci.edu/ml/datasets/arrhyt...,None
4,6,https://archive.ics.uci.edu/ml/datasets/Letter...,https://link.springer.com/article/10.1007%252F...
...,...,...,...
3455,43119,None,None
3456,43121,https://www.kaggle.com/xiaozhouwang/2nd-place-...,None
3457,43122,None,None
3458,43123,https://www4.stat.ncsu.edu/~boos/var.select/di...,https://web.stanford.edu/~hastie/Papers/LARS/L...


In [101]:
#replace the None values with empty lists so the count of string values evaluates to 0
#put item in each cell into list
related_resource_all = related_resource_all.apply(
    lambda row: row.apply(
        lambda cell: [cell] if cell else []
    ),
    axis=1
)
related_resource_all

,did,original_data_url,paper_url
0,[2],[https://archive.ics.uci.edu/ml/datasets/Annea...,[]
1,[3],[https://archive.ics.uci.edu/ml/datasets/Chess...,[https://dl.acm.org/doi/abs/10.5555/32231]
2,[4],[https://archive.ics.uci.edu/ml/datasets/Labor...,[http://citeseerx.ist.psu.edu/viewdoc/summary?...
3,[5],[https://archive.ics.uci.edu/ml/datasets/arrhy...,[]
4,[6],[https://archive.ics.uci.edu/ml/datasets/Lette...,[https://link.springer.com/article/10.1007%252...
...,...,...,...
3455,[43119],[],[]
3456,[43121],[https://www.kaggle.com/xiaozhouwang/2nd-place...,[]
3457,[43122],[],[]
3458,[43123],[https://www4.stat.ncsu.edu/~boos/var.select/d...,[https://web.stanford.edu/~hastie/Papers/LARS/...


In [102]:
#remove 'did' column
related_resource_use = related_resource_all.drop(['did'], axis = 1)
related_resource_use

,original_data_url,paper_url
0,[https://archive.ics.uci.edu/ml/datasets/Annea...,[]
1,[https://archive.ics.uci.edu/ml/datasets/Chess...,[https://dl.acm.org/doi/abs/10.5555/32231]
2,[https://archive.ics.uci.edu/ml/datasets/Labor...,[http://citeseerx.ist.psu.edu/viewdoc/summary?...
3,[https://archive.ics.uci.edu/ml/datasets/arrhy...,[]
4,[https://archive.ics.uci.edu/ml/datasets/Lette...,[https://link.springer.com/article/10.1007%252...
...,...,...
3455,[],[]
3456,[https://www.kaggle.com/xiaozhouwang/2nd-place...,[]
3457,[],[]
3458,[https://www4.stat.ncsu.edu/~boos/var.select/d...,[https://web.stanford.edu/~hastie/Papers/LARS/...


The following code looks at each row of the resulting DataFrame `related_resource_use` and returns `True` if any of the values in it evaluate to `True` (ie not None or empty list)

To break it down,
```python
dd_related_resources.apply(
    lambda row: ...,
    axis=1
)
```
applies the `lambda: row` function to every row in the DataFrame.

```python
lambda row: any(row)
```
is `True` if `any` of the values in the row evaluate as `True`.

In [103]:
related_resource_mask = related_resource_use.apply(
    lambda row: any(row), 
    axis=1
)

related_resource_mask

0        True
1        True
2        True
3        True
4        True
        ...  
3455    False
3456     True
3457    False
3458     True
3459     True
Length: 3460, dtype: bool

In [104]:
print(f'{related_resource_mask.sum()} of the {len(related_resource_all)} objects have a related_resource')

1661 of the 3460 objects have a related_resource


## 23-25. Also, what is the mean number of related resource links per object (again looking at the three properties (Original data URL, Primary manuscript PID/URL, nd Related resource identifier)?
**Property:** Related Resource Identifier

We want to calculate this value to be mean number of links *for objects that have links*

We can start by subsetting `related_resource_use` by `related_resource_mask`

In [105]:
related_resource_use.head()

,original_data_url,paper_url
0,[https://archive.ics.uci.edu/ml/datasets/Annea...,[]
1,[https://archive.ics.uci.edu/ml/datasets/Chess...,[https://dl.acm.org/doi/abs/10.5555/32231]
2,[https://archive.ics.uci.edu/ml/datasets/Labor...,[http://citeseerx.ist.psu.edu/viewdoc/summary?...
3,[https://archive.ics.uci.edu/ml/datasets/arrhy...,[]
4,[https://archive.ics.uci.edu/ml/datasets/Lette...,[https://link.springer.com/article/10.1007%252...


In [106]:
related_resource_mask

0        True
1        True
2        True
3        True
4        True
        ...  
3455    False
3456     True
3457    False
3458     True
3459     True
Length: 3460, dtype: bool

In [107]:
rr_subset = related_resource_use[related_resource_mask]
rr_subset

,original_data_url,paper_url
0,[https://archive.ics.uci.edu/ml/datasets/Annea...,[]
1,[https://archive.ics.uci.edu/ml/datasets/Chess...,[https://dl.acm.org/doi/abs/10.5555/32231]
2,[https://archive.ics.uci.edu/ml/datasets/Labor...,[http://citeseerx.ist.psu.edu/viewdoc/summary?...
3,[https://archive.ics.uci.edu/ml/datasets/arrhy...,[]
4,[https://archive.ics.uci.edu/ml/datasets/Lette...,[https://link.springer.com/article/10.1007%252...
...,...,...
3428,[http://data.dft.gov.uk/road-accidents-safety-...,[https://data.gov.uk/dataset/road-accidents-sa...
3431,[http://www.vincentlemaire-labs.fr/kddcup2009/],[]
3456,[https://www.kaggle.com/xiaozhouwang/2nd-place...,[]
3458,[https://www4.stat.ncsu.edu/~boos/var.select/d...,[https://web.stanford.edu/~hastie/Papers/LARS/...


In [108]:
len(rr_subset)

1661

In [109]:
#function to count links
def count_links(entry):
    try:
        return len(entry)
    except TypeError:
        return 0

In [110]:
links_per_object = rr_subset.apply(
    lambda row: sum([count_links(entry) for entry in row]),
    axis=1
)

links_per_object

0       1
1       2
2       2
3       1
4       2
       ..
3428    2
3431    1
3456    1
3458    2
3459    2
Length: 1661, dtype: int64

In [111]:
print(f'mean {round(links_per_object.mean(), 3)} links per object')

mean 1.142 links per object


In [112]:
print(f'median {round(links_per_object.median(), 3)} links per object')

median 1.0 links per object


## 26. How many objects report each relation type? How many objects report multiple relation types, regardless of what those types are?
**Property:** Related resource relation type

In [113]:
relation_type = df.OpenMLDatasetsCrosswalk.related_resource_relation_type
relation_type

In [114]:
#confirm missing for repo
print(df.OpenMLDatasetsCrosswalk.related_resource_relation_type)

None


## 27. For repositories that store the full citation in a designated field, how many objects have a populated citation? How many objects have a citation and a URL or other actionable link?
**Property:** Citation

In [115]:
citations = df.OpenMLDatasetsCrosswalk.citation
citations

In [116]:
#confirm missing for this repo
print(df.OpenMLDatasetsCrosswalk.citation)

None
